In [ ]:
!pip install pyro-ppl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.2/745.2 kB 10.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
import os

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
import seaborn as sns
import torch

import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam

# fix random generator seed (for reproducibility of results)
np.random.seed(42)

In [ ]:
# load csv
df = pd.read_csv('Data_Train_AllNumbers.csv')
df.head()

,GGGrade,Validation,Yearly_Income,Home_Status,Lend_Amount,Deprecatory_Records,Interest_Charged,Inquiries,Present_Balance,Gross_Collection,Sub_GGGrade,State,Duration,Unpaid_Amount,Reason,Due_Fee,Default
0,1,0,123200,0,10260,0,18.94,0,175923,0,11,0,3,6970,12,0.0,0
1,1,0,255200,2,27916,0,13.27,1,628237,0,13,0,3,17461,12,0.0,1
2,2,2,98842,1,14535,0,15.84,0,66999,0,20,0,3,0,12,0.0,0
3,2,0,149600,0,6840,0,21.80,0,471337,0,19,0,3,7278,12,0.0,0
4,2,0,70400,0,10260,0,20.11,0,166497,0,20,0,3,12967,12,0.0,0


Some descriptive statistics :

In [ ]:
df.describe()

,GGGrade,Validation,Yearly_Income,Home_Status,Lend_Amount,Deprecatory_Records,Interest_Charged,Inquiries,Present_Balance,Gross_Collection,Sub_GGGrade,State,Duration,Unpaid_Amount,Reason,Due_Fee,Default
count,87499.000000,87499.000000,8.749900e+04,87499.000000,87499.000000,87499.000000,87499.000000,87499.000000,8.749900e+04,87499.000000,87499.000000,87499.000000,87499.000000,8.749900e+04,87499.000000,87499.000000,87499.000000
mean,2.810684,1.003280,1.262678e+05,0.591778,25920.735368,0.201419,16.518411,0.670613,2.363910e+05,39.462394,22.024880,22.401547,3.627184,2.791772e+04,2.467651,0.316411,0.188711
std,1.303605,0.778242,1.011390e+05,0.659477,14433.828016,0.577689,4.902028,0.959545,2.550578e+05,391.921413,6.452733,14.915530,0.927911,3.702623e+04,2.394753,3.546108,0.391281
min,1.000000,0.000000,0.000000e+00,0.000000,1710.000000,0.000000,8.000000,0.000000,0.000000e+00,0.000000,11.000000,0.000000,3.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,2.000000,0.000000,7.656000e+04,0.000000,15048.000000,0.000000,12.310000,0.000000,5.173500e+04,0.000000,17.000000,8.000000,3.000000,9.969000e+03,2.000000,0.000000,0.000000
50%,3.000000,1.000000,1.098240e+05,0.000000,23940.000000,0.000000,16.490000,0.000000,1.388070e+05,0.000000,21.000000,22.000000,3.000000,1.961200e+04,2.000000,0.000000,0.000000
75%,4.000000,2.000000,1.584000e+05,1.000000,34200.000000,0.000000,20.770000,1.000000,3.534790e+05,0.000000,26.000000,35.000000,5.000000,3.509600e+04,2.000000,0.000000,0.000000
max,7.000000,2.000000,8.264031e+06,4.000000,59850.000000,22.000000,25.000000,8.000000,5.629234e+06,26308.000000,45.000000,50.000000,5.000000,1.338778e+06,12.000000,169.050000,1.000000


The dataset contains both numerical and categorical features, where:

1. Numerical Features:
Features such as yearly income, lend amount, interest charged, inquiries, etc.

2. Categorical Features:
Features like validation status (e.g., not verified, source verified), home status (e.g., own, mortgage, rent), reason for loan application (e.g., car, credit card, debt consolidation), etc.

Some data preparation before carrying out the model:

In [ ]:
mat = df.values
mat

X = mat[:,0:-1]
print(X)
print(X.shape)


[[1.0000e+00 0.0000e+00 1.2320e+05 ... 6.9700e+03 1.2000e+01 0.0000e+00]
 [1.0000e+00 0.0000e+00 2.5520e+05 ... 1.7461e+04 1.2000e+01 0.0000e+00]
 [2.0000e+00 2.0000e+00 9.8842e+04 ... 0.0000e+00 1.2000e+01 0.0000e+00]
 ...
 [6.0000e+00 2.0000e+00 9.1520e+04 ... 3.7759e+04 1.0000e+00 0.0000e+00]
 [3.0000e+00 0.0000e+00 3.1328e+05 ... 8.8400e+02 0.0000e+00 0.0000e+00]
 [6.0000e+00 2.0000e+00 7.5680e+04 ... 1.3541e+04 0.0000e+00 0.0000e+00]]
(87499, 16)


In [ ]:
y = mat[:,-1].astype("int")
print(y)
print(y.shape)

[0 1 0 ... 0 0 1]
(87499,)


In [ ]:
train_perc = 0.66 # percentage of training data
split_point = int(train_perc*len(y))
perm = np.random.permutation(len(y)) # we also randomize the dataset
ix_train = perm[:split_point]
ix_test = perm[split_point:]
X_train = X[ix_train,:]
print(X_train)
X_test = X[ix_test,:]
y_train = y[ix_train]
print(y_train)
y_test = y[ix_test]
print("num train: %d" % len(y_train))
print("num test: %d" % len(y_test))

[[2.00000e+00 0.00000e+00 7.92000e+04 ... 7.65600e+03 2.00000e+00
  0.00000e+00]
 [5.00000e+00 2.00000e+00 5.72000e+04 ... 0.00000e+00 2.00000e+00
  0.00000e+00]
 [5.00000e+00 1.00000e+00 1.37280e+05 ... 6.59400e+03 2.00000e+00
  0.00000e+00]
 ...
 [2.00000e+00 0.00000e+00 2.63731e+05 ... 3.16080e+04 2.00000e+00
  0.00000e+00]
 [4.00000e+00 2.00000e+00 8.80000e+04 ... 1.61370e+04 2.00000e+00
  0.00000e+00]
 [6.00000e+00 2.00000e+00 9.68000e+04 ... 0.00000e+00 2.00000e+00
  0.00000e+00]]
[0 0 0 ... 0 0 0]
num train: 57749
num test: 29750


In [ ]:
#using sklearn first

# logistic regression model
logreg = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto', C=1)
logreg.fit(X_train, y_train)

# make predictions for test set
y_hat = logreg.predict(X_test)
print("predictions:", y_hat)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

predictions: [0 0 0 ... 0 0 0]
true values: [0 0 0 ... 0 0 0]
Accuracy: 0.8111596638655462


Using Pyro next :


In [ ]:
def model(X, n_cat, obs=None):
    input_dim = X.shape[1]
    alpha = pyro.sample("alpha", dist.Normal(torch.zeros(1, n_cat),
                                             5.*torch.ones(1, n_cat)).to_event())  # Prior for the bias/intercept
    beta  = pyro.sample("beta", dist.Normal(torch.zeros(input_dim, n_cat),
                                            5.*torch.ones(input_dim, n_cat)).to_event()) # Priors for the regression coeffcients

    with pyro.plate("data"):
        y = pyro.sample("y", dist.Categorical(logits=alpha + X.matmul(beta)), obs=obs)

    return y

In [ ]:
n_cat = 2
# Prepare data for Pyro
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()

Run approximate Bayesian inference using SVI:

In [ ]:
# Define guide function
guide = AutoMultivariateNormal(model)

# Reset parameter values
pyro.clear_param_store()

# Define the number of optimization steps
n_steps = 40000

# Setup the optimizer
adam_params = {"lr": 0.001}
optimizer = ClippedAdam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=1)
svi = SVI(model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(X_train, n_cat, y_train)
    if step % 1000 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 4018628212.3
[1000] ELBO: 391951409.9
[2000] ELBO: 355317324.8
[3000] ELBO: 180466713.5
[4000] ELBO: 230463184.1
[5000] ELBO: 65777796.9
[6000] ELBO: 78041535.9
[7000] ELBO: 71759263.2
[8000] ELBO: 67205760.7
[9000] ELBO: 59571512.6
[10000] ELBO: 64985973.4
[11000] ELBO: 53775522.1
[12000] ELBO: 28420579.7
[13000] ELBO: 27565914.3
[14000] ELBO: 12239170.3
[15000] ELBO: 24249126.2
[16000] ELBO: 36925363.7
[17000] ELBO: 11778523.9


In [ ]:
from pyro.infer import Predictive

predictive = Predictive(model, guide=guide, num_samples=2000,
                        return_sites=("alpha", "beta"))
samples = predictive(X_train, n_cat, y_train-1)

In [ ]:
samples_alpha = samples["alpha"].detach().squeeze()
samples_beta = samples['beta'].detach().squeeze()

In [ ]:
alpha_hat = samples_alpha.mean(axis=0).numpy()
beta_hat = samples_beta.mean(axis=0).numpy()

In [ ]:
# make predictions for test set
y_hat = alpha_hat + np.dot(X_test, beta_hat)
y_hat = np.argmax(y_hat, axis=1)
print("predictions:", y_hat)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

predictions: [1 1 0 ... 0 0 0]
true values: [0 0 0 ... 0 0 0]
Accuracy: 0.5683697478991596
